<a href="https://colab.research.google.com/github/hjn14133/Machine-Learning/blob/main/XGB_Regression_Yield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn

from sklearn import preprocessing
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import RFE
from math import log
import matplotlib
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv("FT_join_JH_cleaned.csv")


In [6]:
X = df[['DISTANCE', 'Heading','Elevation', 'AppliedRat']]
y = df['VRYIELDBAL']

# y = df['Loan_Rat']

In [8]:
# Perform data splitting using 80/20 ratio
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Various hyper-parameters to tune
xgb1 = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [3, 4, 5, 6, 7, 8,  9, 10],
              'min_child_weight': [1, 2, 3, 4, 5, 6],
              'silent': [0, 1],
              'subsample': [0.5, 0.75, 1],
              'colsample_bytree': [0.5, 0.7, 1],
              'n_estimators': [500, 1000, 1500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train, Y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 7776 candidates, totalling 15552 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  1.5min
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:  6.5min


In [14]:
parameters = {'objective':['reg:squarederror'],
              'booster':['gbtree', 'gblinear'],
              'learning_rate': [0.1],
              'max_depth': [7,10,15,20],
              'min_child_weight': [10,15,20,25],
              'colsample_bytree': [0.8, 0.9, 1],
              'n_estimators': [300,400,500,600],
              "reg_alpha" : [0.5,0.2,1],
              "reg_lambda" : [2,3,5],
              "gamma" : [1,2,3]}

xgb_model = XGBRegressor(random_state=30)

grid_xgb = RandomizedSearchCV(xgb_model,parameters, cv=5, n_iter=10, scoring='neg_mean_absolute_error', verbose=3, n_jobs=12)

grid_xgb.fit(X_train, Y_train, verbose = 1)

[17:09:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:1.96471
[1]	validation_0-rmse:1.79651
[2]	validation_0-rmse:1.65131
[3]	validation_0-rmse:1.51857
[4]	validation_0-rmse:1.40005
[5]	validation_0-rmse:1.29596
[6]	validation_0-rmse:1.20307
[7]	validation_0-rmse:1.12194
[8]	validation_0-rmse:1.055
[9]	validation_0-rmse:0.991827
[10]	validation_0-rmse:0.93871
[11]	validation_0-rmse:0.889797
[12]	validation_0-rmse:0.851514
[13]	validation_0-rmse:0.812633
[14]	validation_0-rmse:0.779928
[15]	validation_0-rmse:0.75573
[16]	validation_0-rmse:0.729076
[17]	validation_0-rmse:0.70761
[18]	validation_0-rmse:0.691847
[19]	validation_0-rmse:0.673776
[20]	validation_0-rmse:0.65852
[21]	validation_0-rmse:0.645296
[22]	validation_0-rmse:0.633999
[23]	validation_0-rmse:0.62579
[24]	validation_0-rmse:0.619412
[25]	validation_0-rmse:0.613927
[26]	validation_0-rmse:0.609061
[27]	validation_0-rmse:0.600535
[28

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, early_stopping_rounds=10,
             gamma=0, importance_type='gain', learning_rate=0.1,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=None,
             n_estimators=100, n_jobs=-1, nthread=None, objective='reg:linear',
             random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [17]:
predictions = model.predict(X_test)

In [11]:
gb.best_params_

{'colsample_bytree': 1.0,
 'gamma': 0.5,
 'max_depth': 4,
 'min_child_weight': 5,
 'subsample': 0.8}

In [12]:
gb.best_score_

0.6391436530878731